# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [15]:
import pandas as pd

import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.dummy import DummyClassifier

## Откройте и изучите файл

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [5]:
data_train, data_notrain = train_test_split(data, test_size=0.4, random_state=54863)
data_test, data_validate = train_test_split(data_notrain, test_size=0.5, random_state=54863)

In [6]:
print(f'Размер исходной выборки: {data.shape}')
print(f'Размер обучающей выборки: {data_train.shape}')
print(f'Размер валидационной выборки: {data_validate.shape}')
print(f'Размер тестовой выборки: {data_test.shape}')

Размер исходной выборки: (3214, 5)
Размер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выборки: (643, 5)


In [7]:
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

features_validate = data_validate.drop('is_ultra', axis=1)
target_validate = data_validate['is_ultra']

features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

## Исследование моделей

### Logistic Regression

In [8]:

solvers = ['warn']
penalty = ['l2', 'l1']
c_values = [100, 75, 50, 25, 10, 5, 2.0, 1.0, 0.5, 0.25, 0.1, 0.01]

result_lr = pd.DataFrame(columns = ['solver', 'penalty', 'c_value', 'accuracy']) 
for solver in solvers:
    for pen in penalty:
        for c_value in c_values:
            lr_model = LogisticRegression(C=c_value, penalty=pen, solver=solver, random_state=123)
            lr_model.fit(features_train, target_train)
            lr_predict = lr_model.predict(features_validate)
            lr_acc = lr_model.score(features_validate, target_validate) 
            result_lr.loc[len(result_lr)] = [solver, pen, c_value, lr_acc]

lr_max_acc = result_lr['accuracy'].max()
result_lr.query('accuracy==@lr_max_acc')

,solver,penalty,c_value,accuracy
21,warn,l1,0.25,0.734059


### Random Forest

In [9]:

n_estimators = range(1, 100, 3)
max_features = ['sqrt', 'log2']
criterion = ['gini', 'entropy']

result_rf = pd.DataFrame(columns = ['n_estimators', 'max_features', 'criterion', 'accuracy']) 
for estimator in n_estimators:
    for feature in max_features:
        for crit in criterion:
            rf_model = RandomForestClassifier(n_estimators = estimator, max_features = feature, criterion = crit, random_state=123)
            rf_model.fit(features_train, target_train)
            rf_predict = rf_model.predict(features_validate)
            rf_acc = rf_model.score(features_validate, target_validate) 
            result_rf.loc[len(result_rf)] = [estimator, feature, crit, rf_acc]

rf_max_acc = result_rf['accuracy'].max()
result_rf.query('accuracy==@rf_max_acc')

,n_estimators,max_features,criterion,accuracy
40,31,sqrt,gini,0.796267
42,31,log2,gini,0.796267


### K-Nearest Neighbors

In [10]:
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metrics = ['euclidean', 'manhattan', 'minkowski']

result_knn = pd.DataFrame(columns = ['n_neighbors', 'weights', 'metrics', 'accuracy']) 
for n_neighbor in n_neighbors:
    for weight in weights:
        for metric in metrics:
            knn_model = KNeighborsClassifier(metric = metric, weights=weight, n_neighbors = n_neighbor)
            knn_model.fit(features_train, target_train)
            knn_predict = knn_model.predict(features_validate)
            knn_acc = knn_model.score(features_validate, target_validate) 
            result_knn.loc[len(result_knn)] = [n_neighbor, weight, metric, knn_acc]

knn_max_acc = result_knn['accuracy'].max()
result_knn.query('accuracy==@knn_max_acc').head(5)


,n_neighbors,weights,metrics,accuracy
28,9,distance,manhattan,0.768274


<div style="border:solid gray 2.0px; padding: 10px"> 
    
Модель Random Forest с такими параметрами показала наилучший результат на валидационной выборке (accuracy = 0.796)
 - n_estimators = 31
 - criterion = 'gini'
 - max_features: получились две равнозначный модели с разными значениями max_features: sqrt и log2
    
    
    

    На тестовой выборке проверим лучшие модели каждого типа

<div>

## Проверка моделей на тестовой выборке

### Logistic Regression

In [11]:
lr_best_model = LogisticRegression(C=0.25, penalty='l1', solver='warn', random_state=123)
lr_best_model.fit(features_train, target_train)
lr_best_acc = lr_best_model.score(features_test, target_test) 
lr_best_acc

0.7418351477449455

### Random Forest

In [12]:
rf_best_model1 = RandomForestClassifier(n_estimators = 31, max_features = 'sqrt', criterion = 'gini', random_state=123)
rf_best_model1.fit(features_train, target_train)
rf_best_acc1 = rf_best_model1.score(features_test, target_test) 
rf_best_acc1

0.8055987558320373

In [13]:
rf_best_model2 = RandomForestClassifier(n_estimators = 31, max_features = 'log2', criterion = 'gini', random_state=123)
rf_best_model2.fit(features_train, target_train)
rf_best_acc2 = rf_best_model2.score(features_test, target_test) 
rf_best_acc2

0.8055987558320373

### K-Nearest Neighbors

In [14]:
knn_best_model = KNeighborsClassifier(metric = 'manhattan', weights='distance', n_neighbors = 9)
knn_best_model.fit(features_train, target_train)
knn_best_acc = knn_best_model.score(features_test, target_test) 
knn_best_acc

0.7713841368584758

<div style="border:solid gray 2.0px; padding: 10px"> 
    
    На тестовой выборке модель Random Forest так же показала наилучший результат: accuracy = 0.806
    
<div>

## Проверка моделей на адекватность

<div style="border:solid gray 2.0px; padding: 10px"> 
    
    Интересно, что тут имеется ввижу.
    Проверим, что модели хотя бы не предсказывают каждый раз одно и то же значение
    
<div>

In [36]:
lr_best_model = LogisticRegression(C=0.25, penalty='l1', solver='warn', random_state=123)
lr_best_model.fit(features_train, target_train)
lr_best_predict = lr_best_model.predict(features_test)

pd.Series(lr_best_predict).value_counts()

0    591
1     52
dtype: int64

In [37]:
rf_best_model1 = RandomForestClassifier(n_estimators = 31, max_features = 'sqrt', criterion = 'gini', random_state=123)
rf_best_model1.fit(features_train, target_train)
rf_best_predict = rf_best_model1.predict(features_test)

pd.Series(rf_best_predict).value_counts()

0    474
1    169
dtype: int64

In [38]:
knn_best_model = KNeighborsClassifier(metric = 'manhattan', weights='distance', n_neighbors = 9)
knn_best_model.fit(features_train, target_train)
knn_best_predict = knn_best_model.predict(features_test)

pd.Series(knn_best_predict).value_counts()

0    544
1     99
dtype: int64

<div style="border:solid gray 2.0px; padding: 10px"> 
    
    Не понял, в чем суть задания, если честно
    
    Ну, по крайней мере, модели предсказывают не одно и то же значение
    
    
<div>

In [25]:
target_train.value_counts() / target_train.count() * 100

0    69.917012
1    30.082988
Name: is_ultra, dtype: float64

<div style="border:solid blue 2.0px; padding: 10px"> 
    
    Вообще, isultra = 0 (т.е. тариф smart) на тренировочной выборке составлял почти 70% от всей выборке.
    С точки зрения модели, можно было пойти по этому пути и всем предсказывать 0. Получилось бы качество в 70%.

    
    Проверим, что покажет DummyClassifier на разных стратегиях
    
<div>

In [31]:
result_dummy_clf = pd.DataFrame(columns = ['strategy', 'accuracy']) 

for strategy in ["most_frequent", "prior", "stratified", "uniform"]:
    dummy_clf = DummyClassifier(strategy=strategy)
    dummy_clf.fit(features_train, target_train)

    predict = pd.Series(dummy_clf.predict(features_test))
    print(f"Предсказания со стратегией {strategy}:")
    print(predict.value_counts())

    acc_dummy_clf = dummy_clf.score(features_test, target_test)     
    result_dummy_clf.loc[len(result_dummy_clf)] = [strategy, acc_dummy_clf]

result_dummy_clf.head()

Предсказания со стратегией most_frequent:
0    643
dtype: int64
Предсказания со стратегией prior:
0    643
dtype: int64
Предсказания со стратегией stratified:
0    439
1    204
dtype: int64
Предсказания со стратегией uniform:
1    328
0    315
dtype: int64


,strategy,accuracy
0,most_frequent,0.688958
1,prior,0.688958
2,stratified,0.628305
3,uniform,0.528771


<div style="border:solid blue 2.0px; padding: 10px"> 
    
    Стратегии most_frequent и prior пошли именно по этому пути
    
    Наша лучшая модель, что неплохо, выигрывает ))
    
<div>

## Вывод

<div style="border:solid blue 2.0px; padding: 10px"> 
    
Мы попытались настроить разные модели, меняя гиперпараметры, чтобы получить наибольшую точность предсказания.
    
Лучшие модели разных типов:
    
 - **Logistic Regression**:
     - solver = warn
     - penalty = l1
     - c_value = 0.25
     - accuracy на тестовой выборке: 0.741
    
    
 - **Random Forest**:
     - n_estimators = 31
     - max_features = sqrt
     - criterion = gini
     - accuracy на тестовой выборке:  = 0.805

    
- **K-Nearest Neighbors**:
    - n_neighbors = 9
    - weights = distance
    - metrics = manhattan
    - accuracy на тестовой выборке:  = 0.771
    
    
 
-------------------
   
**Лучший результат показала модель Random Forest**
    
   
<div>